In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/gdrive


In [10]:
base_folder = "/content/drive/MyDrive/Colab Notebooks/housing_fall2025"
%cd "{base_folder}"

/content/gdrive/MyDrive/Colab Notebooks/housing_fall2025


In [1]:
import pandas as pd
import sqlite3
import os

def build_churn_database(db_path="project.db"):
    csv_path = "churn_data.csv"
    if not os.path.exists(csv_path):
        csv_path = "../churn_data.csv"
    
    if not os.path.exists(csv_path):
        print("Error: churn_data.csv not found.")
        return

    df = pd.read_csv(csv_path)

    demog_cols = ['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents']
    account_cols = [c for c in df.columns if c not in demog_cols or c == 'customerID']

    if os.path.exists(db_path):
        os.remove(db_path)
    
    conn = sqlite3.connect(db_path)

    df[demog_cols].to_sql('demographics', conn, index=False)
    df[account_cols].to_sql('accounts', conn, index=False)

    query = """
    SELECT d.*, a.* 
    FROM demographics d 
    JOIN accounts a ON d.customerID = a.customerID
    """
    
    df_joined = pd.read_sql(query, conn)
    
    df_joined = df_joined.loc[:, ~df_joined.columns.duplicated()]

    df_joined.to_csv("churn_data.csv", index=False)
    
    if os.path.exists("../docker-compose.yml"):
        df_joined.to_csv("../churn_data.csv", index=False)

    print("Database built.")
    print("SQL Join performed.")
    print("Data saved to churn_data.csv ready for Part 2.")
    print(df_joined.head())

    conn.close()

build_churn_database()

Database built.
SQL Join performed.
Data saved to churn_data.csv ready for Part 2.
   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ... 